In [1]:
import cv2
import numpy as np
# Load HAAR face classifier
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Load functions
def face_extractor(img):
    # Function detects faces and returns the cropped face
    # If no face detected, it returns the input image
    
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    
    if faces is ():
        return None
    
    # Crop all faces found
    for (x,y,w,h) in faces:
        cropped_face = img[y:y+h, x:x+w]

    return cropped_face

# Initialize Webcam
cap = cv2.VideoCapture(0)
count = 0

# Collect 100 samples of your face from webcam input
while True:

    ret, frame = cap.read()
    if face_extractor(frame) is not None:
        count += 1
        face = cv2.resize(face_extractor(frame), (200, 200))
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        # Save file in specified directory with unique name
        file_name_path = './faces/' + 'sathvika_' + str(count) + '.jpg'
        cv2.imwrite(file_name_path, face)

        # Put count on images and display live count
        cv2.putText(face, str(count), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        cv2.imshow('cropped face', face)
        
    else:
        print("Face not found")
        pass

    if cv2.waitKey(1) == 13 or count == 100: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()      
print("---------------------------Collecting Samples  is Completed-----------------------------------------")

<>:14: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:14: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-1-8b035bddff65>:14: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is ():


---------------------------Collecting Samples  is Completed-----------------------------------------


In [2]:
cap.release()

In [3]:
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join


# Get the training data we previously made
data_path = './faces/'
onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path, f))]

# Create arrays for training data and labels
Training_Data, Labels = [], []

# Open training images in our datapath
# Create a numpy array for training data
for i, files in enumerate(onlyfiles):
    image_path = data_path + onlyfiles[i]
    images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    Training_Data.append(np.asarray(images, dtype=np.uint8))
    Labels.append(i)

# Create a numpy array for both training data and labels
Labels = np.asarray(Labels, dtype=np.int32)

# Initialize facial recognizer
# model = cv2.face.createLBPHFaceRecognizer()
# NOTE: For OpenCV 3.0 use cv2.face.createLBPHFaceRecognizer()
# pip install opencv-contrib-python
# model = cv2.createLBPHFaceRecognizer()

vimal_model  = cv2.face_LBPHFaceRecognizer.create()
# Let's train our model 
vimal_model.train(np.asarray(Training_Data), np.asarray(Labels))
print("-------------------Model is trained sucessfully-------------------------")


-------------------Model is trained sucessfully-------------------------


In [5]:
import cv2
import numpy as np
import os
import pywhatkit
import smtplib
import imghdr
import getpass as gp
from email.message import EmailMessage

face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

def face_detector(img, size=0.5):
    
    # Convert image to grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    
    if faces is ():
        return img, []
    
    
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        roi = img[y:y+h, x:x+w]
        roi = cv2.resize(roi, (200, 200))
    return img, roi


# Open Webcam
cap = cv2.VideoCapture(0)

while True:

    ret, frame = cap.read()
    
    image, face = face_detector(frame)
    
    try:
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        # Pass face to prediction model
        # "results" comprises of a tuple containing the label and the confidence value
        results = vimal_model.predict(face)
        # harry_model.predict(face)
        
        if results[1] < 500:
            confidence = int( 100 * (1 - (results[1])/400) )
            display_string = str(confidence) + '% Confident'
            
        cv2.putText(image, display_string, (100, 120), cv2.FONT_HERSHEY_COMPLEX, 1, (255,120,150), 2)
        
        if confidence > 85:
            cv2.putText(image, "Hey Sathvika", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (122,199,122), 2)
            cv2.imshow('Face Recognition', image )
            cv2.imwrite('SathvikaFace.jpg',image)
            print('detected you face...')
            
            #sending whatsapp msg
            print('sending whatsapp message...Please wait...!')
            pywhatkit.sendwhatmsg_instantly(phone_no="+919603569059",
                                 message="Message sent by Face Recognition, Hello...!!!")
            print("Whatsapp Message sent Successfully!!")
            print('sending mail..please wait...!')
            Sender_Email = "satvikakolisetty@gmail.com"
            Reciever_Email = "sathvikakolisetty@gmail.com"
            Password = "1622@Psrgp"
            newMessage = EmailMessage()                         
            newMessage['Subject'] = "A cropped image sent by face recognition app" 
            newMessage['From'] = Sender_Email                   
            newMessage['To'] = Reciever_Email                   
            newMessage.set_content('This a cropped image of detected face!!, image is attached!!') 
            with open('faces/sathvika_1.jpg', 'rb') as f:
                image_data = f.read()
                image_type = imghdr.what(f.name)
                image_name = f.name

            newMessage.add_attachment(image_data, maintype='image', subtype=image_type, filename=image_name)

            with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:

                smtp.login(Sender_Email, Password)              
                smtp.send_message(newMessage)
            print("Mail sent successfully to {} from your mail {}".format(Reciever_Email,Sender_Email)) 
            break
         
        else:
            print("creating key keypair...")
            os.system("aws ec2 create-key-pair --key-name key11221")
            print("Created keypair...")
            print("launching instance, please wait..")
            os.system("aws ec2 run-instances  --image-id ami-0e306788ff2473ccb --instance-type t2.micro  --subnet-id subnet-6bcaf003  --count 1 --security-group-ids sg-019399563be304b5d   --key-name key11221  > ec2Instance.txt")
            print("Instance Launched")
            print("creating volume, please wait..")
            os.system("aws ec2 create-volume --availability-zone ap-south-1a --size 5 --volume-type gp2 --tag-specification ResourceType=volume,Tags=[{Key=Name,Value=volume1}]  > ebsvolume.txt")
            print("Volume Created...")
            print("wait for few minutes instance is being initialized")
            time.sleep(120)
            ec2_id = open("ec2Instance.txt", 'r').read().split(',')[3].split(':')[1].split('"')[1]
            ebs_id = open("ebsvolume.txt", 'r').read().split(',')[6].split(':')[1].split('"')[1]
            os.system("aws ec2 attach-volume --instance-id   " + ec2_id +"  --volume-id  " + ebs_id  +"  --device /dev/xvdf")
            print("Volume Successfully attached to the instance")
            brea

    except:
        cv2.putText(image, "No Face Found", (220, 120) , cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.putText(image, "looking for face", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.imshow('Face Recognition', image )
        pass
        
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break
        
    
    
cap.release()
cv2.destroyAllWindows()     

<>:18: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:18: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-5-7d58bbb1656a>:18: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is ():


creating key keypair...
Created keypair...
launching instance, please wait..
Instance Launched
creating volume, please wait..
Volume Created...
wait for few minutes instance is being initialized
creating key keypair...
Created keypair...
launching instance, please wait..
Instance Launched
creating volume, please wait..
Volume Created...
wait for few minutes instance is being initialized


KeyboardInterrupt: 

In [ ]:
cap.release()